# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

 ```sql
 {{ config(materialized='table') }}


   SELECT
     EXTRACT(YEAR FROM order_approved_at) AS order_year,
     COUNT(DISTINCT order_id) AS orders_count
   FROM
      `dbt-shop-jaffer.dbt_shop.orders`
   WHERE
      order_approved_at IS NOT NULL
   GROUP BY
      1
```

``` shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> dbt run --models orders_by_year 
``` 

![Atividade 01](atividade01_00.JPG)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

```sql
{{ config(materialized='table') }}


   SELECT
      customers.customer_state AS estado,
      COUNT(orders.order_id) AS pedidos_total
   FROM
      `dbt-shop-jaffer.dbt_shop.customers` AS customers
   JOIN
      `dbt-shop-jaffer.dbt_shop.orders` AS orders
   ON
      customers.customer_id = orders.customer_id
   GROUP BY
      customers.customer_state

```

```shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> dbt run --models view_total_pedidos_estados  
```

![Atividade 02](atividade_02_00.JPG)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

```sql
SELECT
      estado,
      pedidos_total
   FROM
      `dbt-shop-jaffer.dbt_shop.view_total_pedidos_estados`
   WHERE
      pedidos_total > {{var('number_of_orders')}}

```

```shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> 
>> dbt run --models view_total_pedidos_estados _retorno_5000 --vars "{number_of_orders': '5000'}"

```

![Atividade 03](atividade_03_00.JPG)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

```sql
SELECT
    CASE
      WHEN customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'Norte'
      WHEN customer_state IN ('MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA') THEN 'Nordeste'
      WHEN customer_state IN ('SP', 'RJ', 'ES', 'MG') THEN 'Sudeste'
      WHEN customer_state IN ('MT', 'MS', 'GO', 'DF') THEN 'Centro Oeste'
      WHEN customer_state IN ('PR', 'RS', 'SC') THEN 'Sul'
    END AS regiao,
    COUNT(*) AS clientes_total
   FROM
      `dbt-shop-jaffer.dbt_shop.customers`
   GROUP BY
      regiao

```

```shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> dbt run --models customers_por_regiao
```

![Atividade 04](atividade_04_00.JPG)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

```sql
{{ config(materialized='table') }}


   SELECT
      *,
      (price + freight_value) AS cost_items
   FROM
      `dbt-shop-jaffer.dbt_shop.items`

```

```shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> dbt run --models items_costs
```

![Atividade 05](atividade_05_00.JPG)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

```sql
{{ config(materialized='table') }}

   SELECT
      cost.order_id,
      order_status,
      (price + freight_value) AS cost_items
   FROM
      `dbt-shop-jaffer.dbt_shop.items_costs` cost
   JOIN
      `dbt-shop-jaffer.dbt_shop.orders` o on cost.order_id = o.order_id
   WHERE
      order_status like "shipped"

```

```shell
(dbt-env) PS C:\Users\jaffe\Desktop\ADA\analytics_engineering\data_analytics_engineering\dbt\shop\models\example> dbt run --models items_costsdbt run --models orders_shipped_costs
```

![Atividade 06](atividade_06_00.JPG)